In [28]:
# imports

# MOD 1: needed to append sys path to import holodeck
import sys
sys.path.append('C:/Users/emiga/OneDrive/Cal/GWs/code/holodeck')


# %load ../init.ipy
%reload_ext autoreload
%autoreload 2
from importlib import reload

import os
import sys
import logging
import warnings
import numpy as np
import astropy as ap
import scipy as sp
import scipy.stats
import matplotlib as mpl
import matplotlib.pyplot as plt

import h5py
import tqdm.notebook as tqdm

import kalepy as kale
import kalepy.utils
import kalepy.plot

import holodeck as holo
import holodeck.sam
from holodeck import cosmo, utils, plot
from holodeck.constants import MSOL, PC, YR, MPC, GYR

# Silence annoying numpy errors
np.seterr(divide='ignore', invalid='ignore', over='ignore')
warnings.filterwarnings("ignore", category=UserWarning)

# Plotting settings
mpl.rc('font', **{'family': 'serif', 'sans-serif': ['Times'], 'size': 15})
mpl.rc('lines', solid_capstyle='round')
mpl.rc('mathtext', fontset='cm')
mpl.style.use('default')   # avoid dark backgrounds from dark theme vscode
plt.rcParams.update({'grid.alpha': 0.5})

log = holo.log
log.setLevel(logging.INFO)

define Nyquist frequency centers and edges

In [29]:
# my code
dur = 5.0*YR/3.1557600
cad = .5*YR/3.1557600
print('dur=%e cad=%e' % (dur,cad))

# standard
fobs = utils.nyquist_freqs(dur,cad)
fobs_edges = utils.nyquist_freqs_edges(dur,cad)
print(f"Number of frequency bins: {fobs.size-1}")
print(f"  between [{fobs[0]*YR:.2f}, {fobs[-1]*YR:.2f}] 1/yr")
print(f"          [{fobs[0]*1e9:.2f}, {fobs[-1]*1e9:.2f}] nHz")

# my code
print('fobs=', fobs)
print('fobs_edges=',fobs_edges) # this seems wrong


print(type(fobs_edges))
fobs_edges_MOD = np.array([1.0e-08, 3.0e-08, 5.0e-08,
         7.0e-08, 9.0e-08, 1.1e-07])
print(fobs_edges_MOD-fobs_edges)

dur=5.000000e+07 cad=5.000000e+06
Number of frequency bins: 4
  between [0.63, 3.16] 1/yr
          [20.00, 100.00] nHz
fobs= [2.e-08 4.e-08 6.e-08 8.e-08 1.e-07]
fobs_edges= [1.0e-08 3.0e-08 5.0e-08 7.0e-08 9.0e-08 1.2e-07]
<class 'numpy.ndarray'>
[ 0.00000000e+00 -6.61744490e-24 -1.32348898e-23  0.00000000e+00
  0.00000000e+00 -1.00000000e-08]


# Semi_Analytic_Model with 
* mtot=[1.0e39, 1.0e40, 1e41] g 
* mrat=[.1, 1]
* redz=[.001, .01, .1, 1]
* gsmf=GSMF_Schechter
* gpf=GPF_Power_Law
* gmt=GMT_Power_Law
* mmbulge=relations.MMBulge_MM2013

In [30]:
my_mtot=(1.0e6*MSOL/1.988409870698051, 
        1.0e8*MSOL/1.988409870698051, 3)
my_mrat=(1e-1, 1.0, 2)
my_redz=(1e-3, 1.0, 4)
my_gsmf=holo.sam.GSMF_Schechter
my_gpf=holo.sam.GPF_Power_Law
my_gmt=holo.sam.GMT_Power_Law
shape=None
my_mmbulge=holo.relations.MMBulge_MM2013
sam = holo.sam.Semi_Analytic_Model(mtot=my_mtot, mrat=my_mrat, 
                redz=my_redz, gsmf=my_gsmf, gpf=my_gpf, gmt=my_gmt, 
                mmbulge=my_mmbulge, shape=my_shape)

in Semi_Analytic_Model()

In [31]:
# Skipping "sanitize input classes/instances"
# Process grid specifications here
# Codes initialization

param_names = ['mtot', 'mrat', 'redz']
params = [my_mtot, my_mrat, my_redz]
for ii, (par, name) in enumerate(zip(params, param_names)):
    log.debug(f"{name}: {par}")
    if isinstance(par, tuple) and (len(par) == 3):
        continue
    elif isinstance(par, np.ndarray):
        continue
    else:
        err = (
            f"{name} (type={type(par)}, len={len(par)}) must be a (3,) tuple specifying a log-spacing, "
            "or ndarray of grid edges!"
        )
        log.exception(err)
        raise ValueError(err)

# Determine shape of grid (i.e. number of bins in each parameter)
if shape is not None:
    if np.isscalar(shape):
        shape = [shape for ii in range(3)]

    shape = np.asarray(shape)
    if not np.issubdtype(shape.dtype, int) or (shape.size != 3) or np.any(shape <= 1):
        raise ValueError(f"`shape` ({shape}) must be an integer, or (3,) iterable of integers, larger than 1!")

    # mtot
    for ii, par in enumerate(params):
        if shape[ii] is not None:
            log.debug(f"{param_names[ii]}: resetting grid shape to {shape[ii]}")
            if not isinstance(par, tuple) or len(par) != 3:
                err = (
                    f"Cannot set shape ({shape[ii]}) for {param_names[ii]} which is not a (3,) tuple "
                    "specifying a log-spacing!"
                )
                log.exception(err)
                raise ValueError(err)

            par = [pp for pp in par]
            par[2] = shape[ii]
            par = tuple(par)
            params[ii] = par

# Set grid-spacing for each parameter
for ii, (par, name) in enumerate(zip(params, param_names)):
    log.debug(f"{name}: {par}")
    if isinstance(par, tuple) and (len(par) == 3):
        par = np.logspace(*np.log10(par[:2]), par[2])
    elif isinstance(par, np.ndarray):
        par = np.copy(par)
    else:
        err = f"{name} must be a (3,) tuple specifying a log-spacing; or ndarray of grid edges!  ({par})"
        log.exception(err)
        raise ValueError(err)

    log.debug(f"{name}: [{par[0]}, {par[-1]}] {par.size}")
    params[ii] = par

my_mtot, my_mrat, my_redz = params

check vars

In [32]:
print(params)
print(np.min(my_mtot-sam.mtot), np.max(my_mtot-sam.mtot))
print(np.min(my_mrat-sam.mrat), np.max(my_mrat-sam.mrat))
print(np.min(my_redz-sam.redz), np.max(my_redz-sam.redz))

[array([1.e+39, 1.e+40, 1.e+41]), array([0.1, 1. ]), array([0.001, 0.01 , 0.1  , 1.   ])]
0.0 0.0
0.0 0.0
0.0 0.0


# Semi_Analytic_Model.GWB
* fobs_gw_edges = fobs_edges
* hard = holo.hardening.Hard_GW
* realize = False

In [33]:
fobs_gw_edges = fobs_edges
hard = holo.hardening.Hard_GW
realize = False
gwb = sam.gwb(fobs_gw_edges, realize=realize)    # calculate many different realizations



before calling hc here is edges:
 [array([1.e+39, 1.e+40, 1.e+41]), array([0.1, 1. ]), array([0.001, 0.01 , 0.1  , 1.   ]), array([5.0e-09, 1.5e-08, 2.5e-08, 3.5e-08, 4.5e-08, 6.0e-08])]
and here is number:
 (2, 1, 3, 5) 
 [[[[0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
    0.00000000e+00]
   [8.82745690e+06 6.46423553e+05 1.44416576e+05 5.00858751e+04
    2.77638151e+04]
   [1.09390698e+09 8.01054307e+07 1.78962414e+07 6.20668998e+06
    3.44051876e+06]]]


 [[[0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
    0.00000000e+00]
   [1.09892126e+05 8.04726202e+03 1.79782747e+03 6.23514038e+02
    3.45628951e+02]
   [1.28993591e+07 9.44603826e+05 2.11032610e+05 7.31893338e+04
    4.05706225e+04]]]]
and realize:
 False
err here
hc:
 [[[[8.52704562e-38 4.29736171e-37 1.10683151e-36 2.16573459e-36
    2.72319684e-36]
   [9.29179258e-40 4.68277002e-39 1.20609755e-38 2.35996821e-38
    2.96742639e-38]
   [1.98216207e-41 9.98947086e-41 2.57289515e-40 5.03437784e-40


In [34]:
# skipped
# squeeze = True
# fobs_gw_edges = np.atleast_1d(fobs_gw_edges)
# if np.isscalar(fobs_gw_edges) or np.size(fobs_gw_edges) == 1:
#     err = "GWB can only be calculated across bins of frequency, `fobs_gw_edges` must provide bin edges!"
#     log.exception(err)
#     raise ValueError(err)

# standard
fobs_gw_cents = kale.utils.midpoints(fobs_gw_edges)
print('fobs_gw_cents', fobs_gw_cents)
# ---- Get the differential-number of binaries for each bin
# convert to orbital-frequency (from GW-frequency)
fobs_orb_edges = fobs_gw_edges / 2.0
fobs_orb_cents = fobs_gw_cents / 2.0

# my code
print('fobs_orb_edges', fobs_orb_edges.shape)
print('fobs_orb_cents', fobs_orb_cents.shape)

fobs_gw_cents [2.00e-08 4.00e-08 6.00e-08 8.00e-08 1.05e-07]
fobs_orb_edges (6,)
fobs_orb_cents (5,)


### straight from dynamic_binary_number

In [35]:
fobs_orb = fobs_orb_cents
hard = hard
from constants import SPLC

# skip
# if (fobs_orb is None) == (sepa is None):
#     err = "one (and only one) of `fobs_orb` or `sepa` must be provided!"
#     log.exception(err)
#     raise ValueError(err)
if fobs_orb is not None:
    fobs_orb = np.asarray(fobs_orb)
    xsize = fobs_orb.size
    my_edges = sam.edges + [fobs_orb, ]

# shape: (M, Q, Z)
my_dens = sam.static_binary_density   # d3n/[dz dlog10(M) dq]  units: [Mpc^-3]

# (Z,) comoving-distance in [Mpc]
dc = cosmo.comoving_distance(sam.redz).to('Mpc').value

# (Z,) this is `(dVc/dz) * (dz/dt)` in units of [Mpc^3/s]
cosmo_fact = 4 * np.pi * (SPLC/MPC) * np.square(dc) * (1.0 + sam.redz)



In [36]:
# (M, Q) calculate chirp-mass
mchirp = utils.m1m2_from_mtmr(sam.mtot[:, np.newaxis], sam.mrat[np.newaxis, :])
mchirp = utils.chirp_mass(*mchirp)
# (M, Q, 1, 1) make shape broadcastable for later calculations
mchirp = mchirp[..., np.newaxis, np.newaxis]

# (M*Q*Z,) 1D arrays of each total-mass, mass-ratio, and redshift
mt, mr, rz = [gg.ravel() for gg in sam.grid]

# Make sure we have both `frst_orb` and binary separation `sa`; shapes (X, M*Q*Z)
if fobs_orb is not None:
    # Convert from observer-frame orbital freq, to rest-frame orbital freq
    # (X, M*Q*Z)
    frst_orb = fobs_orb[:, np.newaxis] * (1.0 + rz[np.newaxis, :])
    sa = utils.kepler_sepa_from_freq(mt[np.newaxis, :], frst_orb)  

In [37]:
# (X, M*Q*Z), hardening rate, negative values, units of [cm/sec]
dadt = hard.dadt(mt[np.newaxis, :], mr[np.newaxis, :], sa)


In [38]:
# Calculate `tau = dt/dlnf_r = f_r / (df_r/dt)`
if fobs_orb is not None:
    # dfdt is positive (increasing frequency)
    dfdt, frst_orb = utils.dfdt_from_dadt(dadt, sa, frst_orb=frst_orb)
    tau = frst_orb / dfdt

In [39]:
limit_merger_time=None
if (limit_merger_time in [None, False]):
    pass
else:
    err = f"`limit_merger_time` ({type(limit_merger_time)}) must be boolean or scalar!"
    log.exception(err)
    raise ValueError(err)

In [40]:
# convert `tau` to the correct shape, note that moveaxis MUST happen _before_ reshape!
# (X, M*Q*Z) ==> (M*Q*Z, X)
tau = np.moveaxis(tau, 0, -1)
# (M*Q*Z, X) ==> (M, Q, Z, X)
tau = tau.reshape(my_dens.shape + (xsize,))

# (M, Q, Z) units: [1/s] i.e. number per second
my_dnum = my_dens * cosmo_fact
# (M, Q, Z, X) units: [] unitless, i.e. number
my_dnum = my_dnum[..., np.newaxis] * tau

bads = ~np.isfinite(tau)
if np.any(bads):
    log.warning(f"Found {utils.frac_str(bads)} invalid hardening timescales.  Setting to zero densities.")
    my_dnum[bads] = 0.0

In [1]:
sams_edges, sams_dnum = sam.dynamic_binary_number(hard, fobs_orb=fobs_orb_cents)
my_edges[-1] = fobs_orb_edges

print('edges\n', len(sams_edges), sams_edges)
print(sams_edges[-1].shape)
print('my_edges\n', len(my_edges), my_edges)
print('my_edges')
print('edges dif min and max\n', 
    np.min(np.array([my_edges])-np.array([sams_edges])),
    np.max(np.array([my_edges])-np.array([sams_edges])))
print('\ndnum\n', dnum)
print('my_dnum\n', my_dnum)
print('dnum dif min and max\n', 
    np.min(np.array([my_dnum])-np.array([dnum])),
    np.max(np.array([my_dnum])-np.array([dnum])))

NameError: name 'sam' is not defined

: 

In [52]:
edges1, dnum1 = sam.dynamic_binary_number(hard, fobs_orb=fobs_orb_edges)
edges2 = [sam.mtot, sam.mrat, sam.redz] + [fobs_orb]
edges3 = [sam.mtot, sam.mrat, sam.redz] + [fobs_orb_edges]

# or
edges4, dnum4 = sam.dynamic_binary_number(hard, fobs_orb=fobs_orb_cents)
edges2 = [sam.mtot, sam.mrat, sam.redz] + [fobs_orb]


print(np.array([edges4]) - np.array([edges2]))
print(np.array([edges1]) - np.array([edges3]))

# THIS IS THE ISSUES!!! Must be edges 3 and 1 or 2 and 4


[[array([0., 0., 0.]) array([0., 0.]) array([0., 0., 0., 0.])
  array([0., 0., 0., 0., 0.])]]
[[array([0., 0., 0.]) array([0., 0.]) array([0., 0., 0., 0.])
  array([0., 0., 0., 0., 0., 0.])]]


In [56]:
sams_edges, samsdnum = sam.dynamic_binary_number(hard, 
                        fobs_orb_cents)
sams_edges[-1] = fobs_orb_edges
sams_number = utils._integrate_grid_differential_number(sams_edges, sams_dnum, freq=False)
sams_number = sams_number * np.diff(np.log(fobs_gw_edges))
log.debug(f"number: {utils.stats(sams_number)}")
log.debug(f"number.sum(): {sams_number.sum():.4e}")
sams_hc = holo.sam.gravwaves._gws_from_number_grid_integrated(sams_edges, sams_number, realize)
print

err here
hc:
 [[[[8.52704562e-38 4.29736171e-37 1.10683151e-36 2.16573459e-36
    2.72319684e-36]
   [9.29179258e-40 4.68277002e-39 1.20609755e-38 2.35996821e-38
    2.96742639e-38]
   [1.98216207e-41 9.98947086e-41 2.57289515e-40 5.03437784e-40
    6.33023175e-40]]]


 [[[1.83709629e-34 9.25838514e-34 2.38459620e-33 4.66593373e-33
    5.86694975e-33]
   [2.00185603e-36 1.00887222e-35 2.59845839e-35 5.08439738e-35
    6.39312636e-35]
   [4.27043872e-38 2.15216625e-37 5.54313457e-37 1.08462383e-36
    1.36380709e-36]]]] 
hc shape: (2, 1, 3, 5)
number:
 [[[[0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
    0.00000000e+00]
   [8.82745690e+06 6.46423553e+05 1.44416576e+05 5.00858751e+04
    2.77638151e+04]
   [1.09390698e+09 8.01054307e+07 1.78962414e+07 6.20668998e+06
    3.44051876e+06]]]


 [[[0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
    0.00000000e+00]
   [1.09892126e+05 8.04726202e+03 1.79782747e+03 6.23514038e+02
    3.45628951e+02]
   [1.28993591e+07